In [1]:
## potrebbe essere interessante parsarlo in un dataframe per avere i risultati come dati tabellari
import json
import os
import sys
import pandas as pd
sys.path.append('../../')

# with open(path, "r") as read_file:
#     data = json.load(read_file)
# 
# print(data)

In [2]:
path = "../great_expectations/uncommitted/validations/Main"
for cartella, sottocartelle, files in os.walk(path):
    for file in files:
        file_path = os.path.join(path, file)
        print(file_path)

../great_expectations/uncommitted/validations/Main\efd6048b7c47bfd83f7ceb6ebe9a8231.json
../great_expectations/uncommitted/validations/Main\97d7545684674b50fe8cbe7c68cd9ad0.json
../great_expectations/uncommitted/validations/Main\97d7545684674b50fe8cbe7c68cd9ad0.json
../great_expectations/uncommitted/validations/Main\97d7545684674b50fe8cbe7c68cd9ad0.json
../great_expectations/uncommitted/validations/Main\0cc71d65a2aa001b140e0176ec2bcefb.json


In [19]:
def parse_json():   
    path = "../great_expectations/uncommitted/validations/Main"
    expectation_type=[]
    expectation_type=[]
    element_count=[]
    missing_count=[]
    missing_percent=[]
    partial_unexpected_counts=[]
    partial_unexpected_index_list=[]
    partial_unexpected_list=[]
    unexpected_count=[]
    unexpected_percent=[]
    unexpected_percent_nonmissing=[]
    unexpected_percent_total=[]
    column=[]
    column_list=[]
    
    for cartella, sottocartelle, files in os.walk(path):
        for file in files:
            #file_path = os.path.join(path, file)
            file_path = path + '/' + file
            with open(file_path, "r") as read_file:
                data = json.load(read_file)
                for e in data['results']:
                    try:
                        expectation_type.append(e['expectation_config']['expectation_type'])
                    except:
                        expectation_type.append(None)
                    try:
                        column.append(e['expectation_config']['kwargs']['column'])
                    except:
                        column.append(e['expectation_config']['kwargs']['column_list'][0])    
                    try:
                        column_list.append(e['expectation_config']['kwargs']['column_list'])
                    except:
                        column_list.append(None)          
                    try:
                        element_count.append(e['result']['element_count'])
                    except:
                        element_count.append(None)
                    try:
                        missing_count.append(e['result']['missing_count'])
                    except:
                        missing_count.append(None)
                    try:
                        missing_percent.append(e['result']['missing_percent'])
                    except:
                        missing_percent.append(None)
                    try:
                        partial_unexpected_counts.append(e['result']['partial_unexpected_counts'])
                    except:
                        partial_unexpected_counts.append(None)
                    try:
                        partial_unexpected_index_list.append(e['result']['partial_unexpected_index_list'])
                    except:
                        partial_unexpected_index_list.append(None)
                    try:
                        partial_unexpected_list.append(e['result']['partial_unexpected_list'])
                    except:
                        partial_unexpected_list.append(None)
                    try:
                        unexpected_count.append(e['result']['unexpected_count'])
                    except:
                        unexpected_count.append(None)
                    try:
                        unexpected_percent.append(e['result']['unexpected_percent'])
                    except:
                        unexpected_percent.append(None)
                    try:
                        unexpected_percent_nonmissing.append(e['result']['unexpected_percent_nonmissing'])
                    except:
                        unexpected_percent_nonmissing.append(None)
                    try:
                        unexpected_percent_total.append(e['result']['unexpected_percent_total'])
                    except:
                        unexpected_percent_total.append(None)
                    try:
                        expectation_type.append(e['expectation_config']['expectation_type'])
                    except:
                        expectation_type.append(None)
                    try:
                        column.append(e['expectation_config']['kwargs']['column'])
                    except:
                        column.append(e['expectation_config']['kwargs']['column_list'][0])    
                    try:
                        column_list.append(e['expectation_config']['kwargs']['column_list'])
                    except:
                        column_list.append(None)          
                    try:
                        element_count.append(e['result']['element_count'])
                    except:
                        element_count.append(None)
                    try:
                        missing_count.append(e['result']['missing_count'])
                    except:
                        missing_count.append(None)
                    try:
                        missing_percent.append(e['result']['missing_percent'])
                    except:
                        missing_percent.append(None)
                    try:
                        partial_unexpected_counts.append(e['result']['partial_unexpected_counts'])
                    except:
                        partial_unexpected_counts.append(None)
                    try:
                        partial_unexpected_index_list.append(e['result']['partial_unexpected_index_list'])
                    except:
                        partial_unexpected_index_list.append(None)
                    try:
                        partial_unexpected_list.append(e['result']['partial_unexpected_list'])
                    except:
                        partial_unexpected_list.append(None)
                    try:
                        unexpected_count.append(e['result']['unexpected_count'])
                    except:
                        unexpected_count.append(None)
                    try:
                        unexpected_percent.append(e['result']['unexpected_percent'])
                    except:
                        unexpected_percent.append(None)
                    try:
                        unexpected_percent_nonmissing.append(e['result']['unexpected_percent_nonmissing'])
                    except:
                        unexpected_percent_nonmissing.append(None)
                    try:
                        unexpected_percent_total.append(e['result']['unexpected_percent_total'])
                    except:
                        unexpected_percent_total.append(None)
                    try:
                        expectation_type.append(e['expectation_config']['expectation_type'])
                    except:
                        expectation_type.append(None)
                    try:
                        column.append(e['expectation_config']['kwargs']['column'])
                    except:
                        column.append(e['expectation_config']['kwargs']['column_list'][0])    
                    try:
                        column_list.append(e['expectation_config']['kwargs']['column_list'])
                    except:
                        column_list.append(None)          
                    try:
                        element_count.append(e['result']['element_count'])
                    except:
                        element_count.append(None)
                    try:
                        missing_count.append(e['result']['missing_count'])
                    except:
                        missing_count.append(None)
                    try:
                        missing_percent.append(e['result']['missing_percent'])
                    except:
                        missing_percent.append(None)
                    try:
                        partial_unexpected_counts.append(e['result']['partial_unexpected_counts'])
                    except:
                        partial_unexpected_counts.append(None)
                    try:
                        partial_unexpected_index_list.append(e['result']['partial_unexpected_index_list'])
                    except:
                        partial_unexpected_index_list.append(None)
                    try:
                        partial_unexpected_list.append(e['result']['partial_unexpected_list'])
                    except:
                        partial_unexpected_list.append(None)
                    try:
                        unexpected_count.append(e['result']['unexpected_count'])
                    except:
                        unexpected_count.append(None)
                    try:
                        unexpected_percent.append(e['result']['unexpected_percent'])
                    except:
                        unexpected_percent.append(None)
                    try:
                        unexpected_percent_nonmissing.append(e['result']['unexpected_percent_nonmissing'])
                    except:
                        unexpected_percent_nonmissing.append(None)
                    try:
                        unexpected_percent_total.append(e['result']['unexpected_percent_total'])
                    except:
                        unexpected_percent_total.append(None)
                    try:
                        expectation_type.append(e['expectation_config']['expectation_type'])
                    except:
                        expectation_type.append(None)
                    try:
                        column.append(e['expectation_config']['kwargs']['column'])
                    except:
                        column.append(e['expectation_config']['kwargs']['column_list'][0])    
                    try:
                        column_list.append(e['expectation_config']['kwargs']['column_list'])
                    except:
                        column_list.append(None)          
                    try:
                        element_count.append(e['result']['element_count'])
                    except:
                        element_count.append(None)
                    try:
                        missing_count.append(e['result']['missing_count'])
                    except:
                        missing_count.append(None)
                    try:
                        missing_percent.append(e['result']['missing_percent'])
                    except:
                        missing_percent.append(None)
                    try:
                        partial_unexpected_counts.append(e['result']['partial_unexpected_counts'])
                    except:
                        partial_unexpected_counts.append(None)
                    try:
                        partial_unexpected_index_list.append(e['result']['partial_unexpected_index_list'])
                    except:
                        partial_unexpected_index_list.append(None)
                    try:
                        partial_unexpected_list.append(e['result']['partial_unexpected_list'])
                    except:
                        partial_unexpected_list.append(None)
                    try:
                        unexpected_count.append(e['result']['unexpected_count'])
                    except:
                        unexpected_count.append(None)
                    try:
                        unexpected_percent.append(e['result']['unexpected_percent'])
                    except:
                        unexpected_percent.append(None)
                    try:
                        unexpected_percent_nonmissing.append(e['result']['unexpected_percent_nonmissing'])
                    except:
                        unexpected_percent_nonmissing.append(None)
                    try:
                        unexpected_percent_total.append(e['result']['unexpected_percent_total'])
                    except:
                        unexpected_percent_total.append(None)
    df_expectations = pd.DataFrame() 
    df_expectations['column'] = column
    df_expectations['column_list'] = column_list
    df_expectations['expectation_type']=expectation_type
    df_expectations['element_count'] = element_count
    df_expectations['missing_count'] = missing_count
    df_expectations['missing_percent'] = missing_percent
    df_expectations['partial_unexpected_counts'] = partial_unexpected_counts
    df_expectations['partial_unexpected_index_list'] = partial_unexpected_index_list
    df_expectations['partial_unexpected_list'] = partial_unexpected_list
    df_expectations['unexpected_count'] = unexpected_count
    df_expectations['unexpected_percent'] = unexpected_percent
    df_expectations['unexpected_percent_nonmissing'] = unexpected_percent_nonmissing
    df_expectations['unexpected_percent_total'] = unexpected_percent_total
    dataframe_type=data['meta']['active_batch_definition']['data_asset_name']
    timestamp_run =data['meta']['batch_markers']['ge_load_time']
    df_expectations['dataframe_type']=dataframe_type
    df_expectations['timestamp_run']=timestamp_run
    df_expectations = df_expectations[ ['dataframe_type'] + ['timestamp_run']+ [ col for col in df_expectations.columns if (col != 'dataframe_type')|(col != 'timestamp_run')] ]
    return df_expectations.iloc[:,0:-2]


In [20]:
parse_json()

FileNotFoundError: [Errno 2] No such file or directory: '../great_expectations/uncommitted/validations/Main/efd6048b7c47bfd83f7ceb6ebe9a8231.json'

In [42]:
path = "../great_expectations/uncommitted/validations/Main"
for cartella, sottocartelle, files in os.walk(path):
    for e in sottocartelle:
        continue
    for file in files:
        print(path + '/' +  e +  '/'+ file)
        file_path = path + '/' +  e +  '/'+ file
        print(file_path)
        with open(file_path, "r") as read_file:
                data = json.load(read_file)

../great_expectations/uncommitted/validations/Main/20230208T121606.507119Z/efd6048b7c47bfd83f7ceb6ebe9a8231.json
../great_expectations/uncommitted/validations/Main/20230208T121606.507119Z/efd6048b7c47bfd83f7ceb6ebe9a8231.json


FileNotFoundError: [Errno 2] No such file or directory: '../great_expectations/uncommitted/validations/Main/20230208T121606.507119Z/efd6048b7c47bfd83f7ceb6ebe9a8231.json'

In [44]:
path = "../great_expectations/uncommitted/validations/Main"
for cartella, files in os.walk(path):
    print
        

ValueError: too many values to unpack (expected 2)